# Low-rank approximation on $\mathcal{P}(d)$ - the space of $d$-dimensional SPD matrices

In this notebook we want to get some intuition in different approaches for computing low-rank approximations for manifold-valued signals

In [2]:
using Manifolds
using Manopt
using LinearAlgebra
using NIfTI
using Plots

In [3]:
include("../../../src/decompositions/naive_HOSVD.jl")
include("../../../src/decompositions/curvature_reweighed_HOSVD.jl")

curvature_reweighed_HOSVD (generic function with 1 method)

### Load data and construct manifold ###

In [4]:
# load data
# ni = niread("data/nifti_dt.nii.gz") * 1e9;
ni = niread("../../../data/nifti_dt_nonlinear.nii.gz") * 1e9;
size(ni)

(112, 112, 50, 1, 6)

In [5]:
 d1, d2, d3, _, d = size(ni)

# construct data as points on the manifold (and add 1e-5 * I for numerical stability)
predata = [ # data ordered as [xx, yx, yy, zx, zy, zz]
    [
    [ni[i,j,k,1,1] + 1e-5;; ni[i,j,k,1,2];; ni[i,j,k,1,4]]; 
    [ni[i,j,k,1,2];; ni[i,j,k,1,3] + 1e-5;; ni[i,j,k,1,5]]; 
    [ni[i,j,k,1,4];; ni[i,j,k,1,5];; ni[i,j,k,1,6] + 1e-5]
    ] for i=1:d1, j=1:d2, k=1:d3];
    
# pick a 2D slice 
data = predata[34:81,32:79,15];

# construct data manifold
n = size(data)
M = SymmetricPositiveDefinite(3)
d = manifold_dimension(M) # -> different than in 1D

6

In [6]:
# Export slice image
asymptote_export_SPD("results/Camino2D_orig.asy", data=data);

### Construct low rank approximation ###

In [20]:
unrolled_data = []
for mat in data
    push!(unrolled_data,mat)
end
q = mean(M, unrolled_data)
# q = 1. * Matrix(I, 3, 3) # TODO compute mean here
log_q_data = log.(Ref(M), Ref(q), data);  # ∈ T_q P(3)^n

(2304,)
[0.7859244247134054 -0.011675282913863529 -0.01643702259871868; -0.0116752829138635 0.7362532545357557 -0.007476745737044835; -0.01643702259871868 -0.007476745737044849 0.7111882776490579]

In [116]:
N_R_q, N_U = naive_HOSVD(M,q, data); 
CR_R_q, CR_U = curvature_reweighed_HOSVD(M,q, data);

(48, 48)
(48,)
(48, 48)
(48,)
288


LoadError: type PowerBasisData has no field eigenvalues

In [ ]:
ref_distance = sqrt(sum(distance.(Ref(M), Ref(q), data).^2))
max_rank = min((size.(N_U)...)...)

N_tangent_distances_r = zeros(max_rank)
N_distances_r = zeros(max_rank)
CR_tangent_distances_r = zeros(max_rank)
CR_distances_r = zeros(max_rank)

for rank in 1:max_rank
    N_log_q_data_r = Symmetric.([sum([N_R_q[i,j] * N_U[1][k,i] * N_U[2][l,j] for i=max_rank-rank+1:max_rank, j=max_rank-rank+1:max_rank]) for k=1:n[1], l=1:n[2]])
    CR_log_q_data_r = Symmetric.([sum([CR_R_q[i,j] * CR_U[1][k,i] * CR_U[2][l,j] for i=max_rank-rank+1:max_rank, j=max_rank-rank+1:max_rank]) for k=1:n[1], l=1:n[2]])
    # expoentiate back
    N_data_r = exp.(Ref(M), Ref(q), N_log_q_data_r)
    CR_data_r = exp.(Ref(M), Ref(q), CR_log_q_data_r)


    # compute relative tangent space error
    N_tangent_distances_r[rank] = sqrt(sum(norm.(Ref(M), Ref(q),  log_q_data - N_log_q_data_r).^2)) / ref_distance
    CR_tangent_distances_r[rank] = sqrt(sum(norm.(Ref(M), Ref(q),  log_q_data - CR_log_q_data_r).^2)) / ref_distance


    # compute relative manifold error
    N_distances_r[rank] = sqrt(sum(distance.(Ref(M), data, N_data_r).^2)) / ref_distance
    CR_distances_r[rank] = sqrt(sum(distance.(Ref(M), data, CR_data_r).^2)) / ref_distance
end

computed first one
computed first one
computed first one
computed first one
computed first one
computed first one
computed first one
computed first one
computed first one
computed first one
computed first one
computed first one
computed first one
computed first one
computed first one
computed first one
computed first one
computed first one
computed first one
computed first one
computed first one
computed first one
computed first one
computed first one
computed first one
computed first one
computed first one
computed first one
computed first one
computed first one
computed first one
computed first one
computed first one
computed first one
computed first one
computed first one
computed first one
computed first one
computed first one
computed first one
computed first one
computed first one
computed first one
computed first one
computed first one
computed first one
computed first one
computed first one


In [ ]:
plot(1:max_rank, [N_distances_r, CR_distances_r], label = ["standard inner product" "reweighted inner product"], ylims=(0,1), xlims=(1,max_rank))
savefig("results/DTI_camino_2D_rank.png")

"/Users/wdiepeveen/Documents/PhD/Projects/8 - Manifold-valued tensor decomposition/src/manifold-valued-tensors/experiments/2D/P3/results/DTI_camino_1D_rank.png"